In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Desktop\\E2E project\\ML-wine-project-End-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class  DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_pathway=CONFIG_FILE_PATH, 
                 param_pathway=PARAM_FILE_PATH, 
                 schema_pathway=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_pathway)
        self.param = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

    


In [7]:
# import os
from mlproject.logging import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os


In [10]:



class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        # Load data
        data = pd.read_csv(self.config.data_path)

        # Split into train and test sets
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        # Identify the features and target
        features = [col for col in data.columns if col != 'quality']
        # target = 'quality'

        # Initialize StandardScaler
        scaler = StandardScaler()

        # Scale the features in the train and test sets
        train[features] = scaler.fit_transform(train[features])
        test[features] = scaler.transform(test[features])

        # Save the scaled train and test sets
        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        # Logging
        logger.info("Split into training and test sets, with scaling applied.")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        # Print the shapes (optional)
        print(f"Train shape: {train.shape}")
        print(f"Test shape: {test.shape}")


In [12]:
#pipeline

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()

except Exception as e:
    raise e

[2024-12-24 23:22:12,635: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-24 23:22:12,636: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-24 23:22:12,641: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-24 23:22:12,642: INFO: common: created directory at: artifacts]
[2024-12-24 23:22:12,644: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-24 23:22:12,689: INFO: 1202428394: Split into training and test sets, with scaling applied.]
[2024-12-24 23:22:12,691: INFO: 1202428394: Train shape: (914, 13)]
[2024-12-24 23:22:12,692: INFO: 1202428394: Test shape: (229, 13)]
Train shape: (914, 13)
Test shape: (229, 13)
